# Initial Test
Here, we can open a file and parameterize with a small molecule, as long as the hydrogens have been added manually and provide a smiles string.

In [7]:
from openff.toolkit import Molecule
molecule = Molecule.from_smiles('c1ccccc1')

from openmmforcefields.generators import GAFFTemplateGenerator
gaff = GAFFTemplateGenerator(molecules=molecule)

from openmm.app import ForceField
forcefield = ForceField('amber/protein.ff14SB.xml', 'amber/tip3p_standard.xml', 'amber/tip3p_HFE_multivalent.xml')

forcefield.registerTemplateGenerator(gaff.generator)

#from openmm.app import PDBFile
#pdbfile = PDBFile('4W52_h.pdb')

from pdbfixer import PDBFixer

#fix it:
pdb = PDBFixer(str('4W52_h.pdb'))
### Fix messed up residues
pdb.findMissingResidues()
#pdb.findNonstandardResidues()
#pdb.replaceNonstandardResidues() 
pdb.findMissingAtoms()
pdb.addMissingAtoms()    #this adds both missing atoms and residues 

#add hydrogens:
pdb.addMissingHydrogens(7.0)

#print the names of all non-standard residues:
standard_residues = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL', 'HOH']
for residue in pdb.topology.residues():
    if residue.name not in standard_residues:
        print(residue.name)

#make a Modeller object:
from openmm.app import Modeller
modeller = Modeller(pdb.topology, pdb.positions)

#delete EPE:
to_delete = []
for residue in modeller.topology.residues():
    if residue.name == 'EPE':
        to_delete.append(residue)
modeller.delete(to_delete)

#system = forcefield.createSystem(pdb.topology)
system = forcefield.createSystem(modeller.topology)

BNZ
EPE


In [16]:
#integrator:
from openmm import LangevinIntegrator
from openmm import unit
integrator = LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 2*unit.femtoseconds)

#simulation:
from openmm.app import Simulation
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

#minimize:
simulation.minimizeEnergy()
from pathlib import Path
input_pdb_file = '4W52_h.pdb'
PDBFile.writeFile(simulation.topology, simulation.context.getState(getPositions=True).getPositions(), str( (str(Path(input_pdb_file).stem) + '_minim.pdb')))

#report:
from openmm.app import PDBFile
from openmm.app import DCDReporter
reporting_interval = 10
simulation.reporters.append(DCDReporter((str(Path(input_pdb_file).stem) + '_equil.dcd'), reporting_interval))

#run:
simulation.step(1000)

In [27]:
import mdtraj as md
traj = md.load(
    str((str(Path(input_pdb_file).stem) + '_equil.dcd')),
    top = str( (str(Path(input_pdb_file).stem) + '_minim.pdb'))            #get topology from processed pdb
)

#center the trajectory around the protein (to stop periodic boundary artifacts)
protein = traj.topology.guess_anchor_molecules()
centered_traj = traj.image_molecules(anchor_molecules=protein)

#first let's make copy of trajectory without water:
traj_no_water = centered_traj.atom_slice(centered_traj.topology.select('not water'))

import nglview as nv
view = nv.show_mdtraj(traj_no_water)
view.add_representation('cartoon', selection="protein")
view._set_size('800px', '800px')
#view.add_representation('ball+stick', selection='water', opacity=0.1)
view

#test!

NGLWidget(max_frame=99)

# Extracting the small molecule from the PDB:
Let's do the same as above, but extract it using just the residue name or number:

In [100]:
import openmm

def create_ligand_topology(pdb, ligand_residue_idx_chain):
    #First, find the ligand:
    ligand = None
    ligand_name = None
    for residue in pdb.topology.residues():
        if int(residue.id) == ligand_residue_idx_chain[0] and int(residue.chain.index) == ligand_residue_idx_chain[1]:   #note: residue.index is NOT the same as residue.id!
            ligand = residue
            ligand_name = residue.name
            break
    if ligand is None:
        raise ValueError(f'Could not find ligand with residue index {ligand_residue_idx_chain}')
    print(f'Found ligand: {ligand.name} with {len(list(ligand.atoms()))} atoms')

    #create a new topology object:
    ligand_topology = openmm.app.Topology()

    #add all atoms to a new residue and chain:
    ligand_chain = ligand_topology.addChain()
    ligand_residue = ligand_topology.addResidue(ligand.name, ligand_chain)
    [ligand_topology.addAtom(atom.name, atom.element, ligand_residue) for atom in ligand.atoms()]

    #get positions from ligand.atoms():
    ligand_positions = [pdb.positions[atom.index] for atom in ligand.atoms()]
    ligand_positions = [pos.value_in_unit(openmm_unit.angstroms) for pos in ligand_positions]

    return ligand_topology, ligand_positions, ligand_name

import requests
def download_ligand_sdf(three_letter_code, file_path):
    url = f'https://files.rcsb.org/ligands/download/{three_letter_code}_ideal.sdf'
    response = requests.get(url)
    
    if response.status_code == 200:
        with open(file_path, 'w') as file:
            file.write(response.text)
        print(f"SDF file for {three_letter_code} downloaded successfully.")
    else:
        print(f"Failed to download SDF file. HTTP status code: {response.status_code}")

In [101]:
ligand_name = 'EPE'

### FIX PDB FILE:
# fix messed up residues
from pdbfixer import PDBFixer
pdb = PDBFixer(str('4W52_h.pdb'))
pdb.findMissingResidues()
pdb.findNonstandardResidues()
pdb.replaceNonstandardResidues() 
pdb.findMissingAtoms()
pdb.addMissingAtoms()    #this adds both missing atoms and residues 

#add hydrogens:
pdb.addMissingHydrogens(7.0)

#print the names of all non-standard residues:
standard_residues = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']
nonstandard_residues = []
for residue in pdb.topology.residues():
    if residue.name not in standard_residues:
        nonstandard_residues.append(residue.name)


### MAKE MODELLER OBJECT:
from openmm.app import Modeller
modeller = Modeller(pdb.topology, pdb.positions)

#delete all nonstandard residues except ligand:
to_delete = []
for residue in modeller.topology.residues():
    if residue.name in nonstandard_residues and residue.name != ligand_name:
        to_delete.append(residue)
modeller.delete(to_delete)

#write out the cleaned PDB file:
from openmm.app import PDBFile
PDBFile.writeFile(modeller.topology, modeller.positions, str('cleaned.pdb'))


In [128]:
### EXTRACT LIGAND TO OPENFF MOLECULE:
from openff.toolkit import Molecule
# what goes here? lol

### MAKE LIGAND AND SYSTEM:
from openff.toolkit import Molecule
#first, extract ligand from the PDB file:

# get ligand idx from residue name:
ligand_residue_idx_chain = None
for residue in pdb.topology.residues():
    if residue.name == ligand_name:
        ligand_residue_idx_chain = int(residue.id), int(residue.chain.index)
        break

#Let's take the original pdb file, once again:
original_pdb = pdb

#get the ligand topology and positions:
ligand_topology, ligand_positions, ligand_code = create_ligand_topology(original_pdb, ligand_residue_idx_chain)

#write the ligand to a new PDB file:
ligand_path_pdb = 'ligand.pdb'
PDBFile.writeFile(ligand_topology, ligand_positions, str(ligand_path_pdb))

#Then, write to SDF file, making sure to keep hydrogens:
ligand_mol = Chem.MolFromPDBFile(str(ligand_path_pdb), removeHs=False)

#specify the stereochemistry:
Chem.AssignStereochemistryFrom3D(ligand_mol)

ligand_path = 'ligand.sdf'
writer = Chem.SDWriter(str(ligand_path))
writer.write(ligand_mol)
writer.close()


#get ligand mol:
ligand_mol = Chem.MolFromMolFile(ligand_path, removeHs=False)

#load the template molecule:
template_path = 'ligand_template.sdf'
download_ligand_sdf(ligand_name, template_path)
template_mol = Chem.MolFromMolFile(template_path, removeHs=False)


template_mol = Chem.RemoveHs(template_mol)
ligand_mol = Chem.RemoveHs(ligand_mol)

#align the ligand to the template:
from rdkit.Chem import AllChem
mol = AllChem.AssignBondOrdersFromTemplate(template_mol, ligand_mol)

#turn into openff molecule:
ligand = Molecule.from_rdkit(mol)

from openmmforcefields.generators import GAFFTemplateGenerator
gaff = GAFFTemplateGenerator(molecules=ligand)

from openmm.app import ForceField
forcefield = ForceField('amber/protein.ff14SB.xml', 'amber/tip3p_standard.xml', 'amber/tip3p_HFE_multivalent.xml')

forcefield.registerTemplateGenerator(gaff.generator)

#system = forcefield.createSystem(pdb.topology)
system = forcefield.createSystem(modeller.topology)



Found ligand: EPE with 33 atoms
SDF file for EPE downloaded successfully.


[13:55:59] WARNING: More than one matching pattern found - picking one



In [129]:
#integrator:
from openmm import LangevinIntegrator
from openmm import unit
integrator = LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 2*unit.femtoseconds)

#simulation:
from openmm.app import Simulation
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

#minimize:
simulation.minimizeEnergy()
from pathlib import Path
from openmm.app import PDBFile
input_pdb_file = '4W52_h.pdb'
PDBFile.writeFile(simulation.topology, simulation.context.getState(getPositions=True).getPositions(), str( (str(Path(input_pdb_file).stem) + '_minim.pdb')))

#report:
from openmm.app import PDBFile
from openmm.app import DCDReporter
reporting_interval = 10
simulation.reporters.append(DCDReporter((str(Path(input_pdb_file).stem) + '_equil.dcd'), reporting_interval))

#run:
simulation.step(1000)

In [130]:
#show with nglview:
import mdtraj as md
traj = md.load(
    str((str(Path(input_pdb_file).stem) + '_equil.dcd')),
    top = str( (str(Path(input_pdb_file).stem) + '_minim.pdb'))            #get topology from processed pdb
)

#center the trajectory around the protein (to stop periodic boundary artifacts)
#select the protein:
#protein = traj.topology.guess_anchor_molecules()
#centered_traj = traj.image_molecules(anchor_molecules=protein)

#first let's make copy of trajectory without water:
#traj_no_water = centered_traj.atom_slice(centered_traj.topology.select('not water'))

import nglview as nv
view = nv.show_mdtraj(traj)
view.add_representation('cartoon', selection="protein")
view._set_size('800px', '800px')
#view.add_representation('ball+stick', selection='water', opacity=0.1)
view

NGLWidget(max_frame=99)